## Long Term Memory

In [17]:
MISTRALAI_API_KEY ="XN1gUFU0bOnw0nDGERkN37MAOHwc4WdQ"
import os
if not os.getenv("MISTRALAI_API_KEY"):
    os.environ["MISTRALAI_API_KEY"] = MISTRALAI_API_KEY

In [22]:
!pip install langgraph

In [38]:


from deepagents import create_deep_agent
from deepagents.backends import CompositeBackend, StateBackend, StoreBackend
from langgraph.store.memory import InMemoryStore
from langgraph.checkpoint.memory import MemorySaver
from langchain_mistralai import ChatMistralAI



composite_backend = lambda rt: CompositeBackend(
    default=StateBackend(rt),
    routes={
        "/memories/": StoreBackend(rt),
    }
)


llm = ChatMistralAI(
    model="mistral-large-latest",
    api_key= MISTRALAI_API_KEY,
    temperature=0,
    max_retries=2,
    max_tokens=2000
    # other params...
)   

agent = create_deep_agent(
    backend=composite_backend,
    store=InMemoryStore() , # Store passed to create_deep_agent, not backend
    model= llm
)



# checkpointer = MemorySaver()

# def make_backend(runtime):
#     return CompositeBackend(
#         default=StateBackend(runtime),  # Ephemeral storage
#         routes={
#             "/memories/": StoreBackend(runtime)  # Persistent storage
#         }
#     )



 

# agent = create_deep_agent(
#     model=llm,
#     store=InMemoryStore(),  # Required for StoreBackend
#     backend=make_backend,
#     checkpointer=checkpointer
# )

In [43]:
from deepagents import create_deep_agent
from deepagents.backends import CompositeBackend, StateBackend, FilesystemBackend

composite_backend = lambda rt: CompositeBackend(
    default=StateBackend(rt),
    routes={
        "/memories/": FilesystemBackend(root_dir="/deepagents/myagent", virtual_mode=True),
    },
)

agent = create_deep_agent(
    model=llm,
    store=InMemoryStore(),  # Required for StoreBackend
    backend=composite_backend
)

In [44]:
config1 = {"configurable": {"thread_id": "salman1"}}


result = agent.invoke({"messages": [{"role": "user", "content": "My Name is Salman, I love tea"}]},config=config1)


In [45]:
result

{'messages': [HumanMessage(content='My Name is Salman, I love tea', additional_kwargs={}, response_metadata={}, id='c9fad74c-4d71-4bf8-b3b7-7e1be9092aa7'),
  AIMessage(content="Hello Salman! 😊 It's great to meet you! What’s your favorite kind of tea? I’d love to hear about it—whether it’s a cozy chamomile, a bold chai, or something else entirely! 🍵✨", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 4962, 'total_tokens': 5019, 'completion_tokens': 57}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'stop', 'model_provider': 'mistralai'}, id='lc_run--019bb32f-8de9-78d0-a153-478a7acfed22-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 4962, 'output_tokens': 57, 'total_tokens': 5019})]}

In [46]:
result2 = agent.invoke({"messages": [{"role": "user", "content": "What is my name ?"}]},config=config1)

In [47]:
result2

{'messages': [HumanMessage(content='What is my name ?', additional_kwargs={}, response_metadata={}, id='18fbacd7-8163-4a99-ba88-5e0ad33db534'),
  AIMessage(content='I don’t have access to your name unless you’ve shared it with me in our conversation. Could you remind me? 😊', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 4958, 'total_tokens': 4986, 'completion_tokens': 28}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'stop', 'model_provider': 'mistralai'}, id='lc_run--019bb32f-c42f-7791-9623-1184c342a4a2-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 4958, 'output_tokens': 28, 'total_tokens': 4986})]}

In [28]:
# Transient file (lost after thread ends)
# agent.invoke({
#     "messages": [{"role": "user", "content": "Write draft to /draft.txt"}]
# },config=config1)

# Persistent file (survives across threads)
agent.invoke({
    "messages": [{"role": "user", "content": "Save final report to /memories/report.txt"}]
},config=config1)

{'messages': [HumanMessage(content='Save my preferences to /memories/preferences.txt', additional_kwargs={}, response_metadata={}, id='31be88c9-b726-4ee3-b0b6-49ee71a961a1'),
  AIMessage(content="I'll save your preferences to `/memories/preferences.txt`. Since this is a new file, I'll use the `write_file` tool to create it.\n\nWhat preferences would you like me to save? For example:\n- Theme (light/dark mode)\n- Notification settings\n- Default view or layout\n- Any other specific preferences you'd like to store.", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 4964, 'total_tokens': 5040, 'completion_tokens': 76}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'stop', 'model_provider': 'mistralai'}, id='lc_run--019bb327-dafe-79f1-bac5-c905cb59b261-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 4964, 'output_tokens': 76, 'total_tokens': 5040}),
  HumanMessage(content='Save final report to /mem

In [29]:
import uuid

# Thread 1: Write to long-term memory
config1 = {"configurable": {"thread_id": str(uuid.uuid4())}}
agent.invoke({
    "messages": [{"role": "user", "content": "Save my preferences to /memories/preferences.txt"}]
}, config=config1)

# Thread 2: Read from long-term memory (different conversation!)
config2 = {"configurable": {"thread_id": str(uuid.uuid4())}}
agent.invoke({
    "messages": [{"role": "user", "content": "What are my preferences?"}]
}, config=config2)


{'messages': [HumanMessage(content='What are my preferences?', additional_kwargs={}, response_metadata={}, id='7a27a1dd-824c-4fdd-8a90-0ecdde99a74c'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'xoo67662m', 'function': {'name': 'ls', 'arguments': '{"path": "/"}'}, 'index': 0}]}, response_metadata={'token_usage': {'prompt_tokens': 4958, 'total_tokens': 4967, 'completion_tokens': 9}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls', 'model_provider': 'mistralai'}, id='lc_run--019bb329-45bd-7b21-97ed-88a5957ce28a-0', tool_calls=[{'name': 'ls', 'args': {'path': '/'}, 'id': 'xoo67662m', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 4958, 'output_tokens': 9, 'total_tokens': 4967}),
  ToolMessage(content="['/memories/']", name='ls', id='af199724-7b4f-4ae9-b177-a07be307bd0e', tool_call_id='xoo67662m'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'dqH2feNwX', 'function'